# Collision tutorial

we'll demonstrate self collision on a sawyer robot

You can see that arm of the sawyer robot collide with milk object.

## Setup
If you want to check robot's collision, install python-fcl 

And then, import CollisionManager

In [1]:
import numpy as np
import trimesh
import os

from pykin.robots.single_arm import SingleArm
from pykin.kinematics.transform import Transform
from pykin.collision.collision_manager import CollisionManager
from pykin.utils.kin_utils import apply_robot_to_scene
from pykin.utils.kin_utils import ShellColors as sc

In [2]:
file_path = 'urdf/sawyer/sawyer.urdf'
robot = SingleArm(file_path, Transform(rot=[0.0, 0.0, 0.0], pos=[0, 0, 0.913]))
robot.setup_link_name("sawyer_base", "sawyer_right_hand")

## Use CollisionManager

Create robot's collision manager using CollisionManager

And check self collision

In [3]:
c_manager = CollisionManager(is_robot=True)
c_manager.setup_robot_collision(robot, geom="visual")
c_manager.show_collision_info()

goal_qpos = np.array([0, 0, 0, 0, 0, 0, 0, 0])
robot.set_transform(goal_qpos)

******************** Robot Collision Info ********************
sawyer_pedestal [ 0.25999999  0.345      -0.00188   ]
sawyer_right_arm_base_link [0.         0.         0.91299999]
sawyer_link_0 [0.         0.         0.99299997]
sawyer_head [0.     0.     1.2895]
sawyer_screen [0.03       0.         1.39450002]
sawyer_link_1 [0.081      0.05       1.23000002]
sawyer_link_2 [0.221      0.1925     1.23000002]
sawyer_link_3 [0.48100001 0.1505     1.23000002]
sawyer_link_4 [0.60600001 0.024      1.23000002]
sawyer_link_5 [0.88099998 0.055      1.23000002]
sawyer_link_6 [0.991      0.1603     1.23000002]
sawyer_torso [0.         0.         0.91299999]
***************************************************************



In [4]:
for link, info in robot.info[c_manager.geom].items():
    if link in c_manager._objs:
        c_manager.set_transform(name=link, h_mat=info[3])

current_file_path = os.path.abspath('')
milk_path = current_file_path + "/../pykin/assets/objects/meshes/milk.stl"
test_mesh = trimesh.load_mesh(milk_path)

o_manager = CollisionManager()
o_manager.add_object("milk1", gtype="mesh", gparam=test_mesh, h_mat=Transform(pos=[0.1, 0, 0.4]).h_mat)
o_manager.add_object("milk2", gtype="mesh", gparam=test_mesh, h_mat=Transform(pos=[0.4, 0, 0.4]).h_mat)

scene = trimesh.Scene()
scene = apply_robot_to_scene(trimesh_scene=scene, robot=robot, geom=c_manager.geom)
# scene.set_camera(np.array([np.pi/2, 0, np.pi/2]), 5, resolution=(1024, 512))

scene.add_geometry(test_mesh, node_name="milk1", transform=Transform(pos=[0.1, 0, 0.4]).h_mat)
scene.add_geometry(test_mesh, node_name="milk2", transform=Transform(pos=[0.4, 0, 0.4]).h_mat)

'milk2'

Check collision between robot and objects

In [5]:
result, name = c_manager.in_collision_other(o_manager, return_names=True)

if result:
    print(f"{sc.FAIL}Collide!! {sc.ENDC}{list(name)[0][0]} and {list(name)[0][1]}")
scene.show()

Collide!! sawyer_pedestal and milk1


/opt/homebrew/lib/python3.9/site-packages/IPython/core/display.py:414: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
